#### Tank 22MT02  - 20 tonne tank 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

C:\Users\User\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
uploaded = pd.read_csv('Bev Base Dearation Data Extract .csv')
# Dataset is now stored in a Pandas Dataframe

In [3]:
#uploaded.head()

In [4]:
# Or rename the existing DataFrame (rather than creating a copy)
uploaded.rename(columns={'SKU': 'Material','Deaeration Phase':'Deaeration_Phase'}, inplace=True)

In [5]:
# Removal of unwanted columns from dataset
uploaded1=uploaded.drop(['SKU_DESCRIPTION', 'Recipe', 'Tank','Source', 'Batch_Phase_Type','Phase_start','Phase_end','Deaeration_Phase','Phase_row_no','Instruction_Step','Material_Name','Target_Phase_duration','Target_Flowrate'], axis=1)

In [6]:
# Define a dictionary of replacements
replacements = {
    'U_25MT02': '2502',
    'U_25MT03': '2503',
    'U_25MT04': '2504',
    'U_26MT01' : '2601',
    'U_23MT02' : '2302',
    'U_23MT04' : '2304',
    'U_25MT01' : '2501',
    'U_23MT05' : '2305',
    'U_25MT02' : '2502',
    'U_22MT04' : '2204',
    'U_22MT03' : '2203',
    'U_22MT02' : '2202',
    'U_26MT04' : '2604',
    'U_26MT03' : '2603',
    'U_23MT03' : '2303',
    'U_22MT05' : '2205'
}

# Replace values in the 'TankName' column
uploaded1['Unit'].replace(replacements, inplace=True)

# Print the modified DataFrame
#print(uploaded1)

In [7]:
#uploaded1.info()

In [45]:
#Select the batches where 22MT03 tanks was used
Unit = ['2202']
TANK_2202= uploaded1[uploaded1['Unit'].isin(Unit)]

#print(TANK_2202)

In [53]:
#Aggregate data per tank
aggregated_df_TANK_2202= TANK_2202.groupby('Material').agg({
    'Phase_duration': 'mean',
    'Phase_overrun': 'sum',
    'Phase_start_delay':'sum',
    'Quantity':'sum',
    'Flowrate_KGMIN':'sum',
    
}).reset_index()

 #Print the aggregated DataFrame
print(aggregated_df_TANK_2202)

   Material  Phase_duration  Phase_overrun  Phase_start_delay    Quantity  \
0   1397022       31.285714          477.0               4730   64767.364   
1   1428047       40.279070         1260.0               6172   69013.570   
2   1451704       45.947368          667.0               4254   24377.066   
3   1451710       19.777778           17.0                 42   17356.717   
4   1520984       60.809524          856.0               3295  131440.377   
5   1648637       35.897959         1177.0               6329   72853.921   
6   1775253       48.063830         2976.0              17069  184005.886   

   Flowrate_KGMIN  
0       1910.7127  
1       3739.7013  
2       1306.4656  
3        192.8524  
4       2220.3772  
5       3251.5634  
6       6803.4279  


In [54]:
aggregated_df_TANK_2202.describe()

,Material,Phase_duration,Phase_overrun,Phase_start_delay,Quantity,Flowrate_KGMIN
count,7.000000e+00,7.000000,7.000000,7.000000,7.000000,7.000000
mean,1.524765e+06,40.294463,1061.428571,5984.428571,80544.985857,2775.014357
std,1.381125e+05,13.131435,944.151622,5325.021028,58995.975317,2131.692251
min,1.397022e+06,19.777778,17.000000,42.000000,17356.717000,192.852400
25%,1.439876e+06,33.591837,572.000000,3774.500000,44572.215000,1608.589150
50%,1.451710e+06,40.279070,856.000000,4730.000000,69013.570000,2220.377200
75%,1.584810e+06,47.005599,1218.500000,6250.500000,102147.149000,3495.632350
max,1.775253e+06,60.809524,2976.000000,17069.000000,184005.886000,6803.427900


In [55]:
aggregated_df_TANK_2202.corr()

,Material,Phase_duration,Phase_overrun,Phase_start_delay,Quantity,Flowrate_KGMIN
Material,1.000000,0.327347,0.837479,0.800428,0.749077,0.783047
Phase_duration,0.327347,1.000000,0.465301,0.364357,0.646050,0.418402
Phase_overrun,0.837479,0.465301,1.000000,0.977063,0.843250,0.981671
Phase_start_delay,0.800428,0.364357,0.977063,1.000000,0.794516,0.961120
Quantity,0.749077,0.646050,0.843250,0.794516,1.000000,0.838957
Flowrate_KGMIN,0.783047,0.418402,0.981671,0.961120,0.838957,1.000000


In [50]:
Material=TANK_2202.Material.value_counts()
Material

1775253    94
1648637    49
1428047    43
1520984    42
1397022    35
1451704    19
1451710     9
Name: Material, dtype: int64

### Creation Dataframes based on the ingredients addition - quantity , flow rates , duration times , overrun times and targets for each 

### Ingredients addition

## Treated Water- 2 phase additions with different quantities
MAjority of additions are treated water bulk additions 
NB : Need to separate out the water additions

In [11]:
#Select all the rows where the phase HP is in the list below - HP is homogenisation of the final batch quantity
Material_ID = ['1002565']
df_IDW_2202 = TANK_2202[TANK_2202['Material_ID'].isin(Material_ID)]
print(df_IDW_2202)

      Material    BATCHID  Unit Material_ID   Quantity  Phase_duration  \
2002   1397022  107956670  2202     1002565   8623.000              10   
2012   1397022  107956670  2202     1002565   2065.000              13   
2755   1428047  107993270  2202     1002565   5101.000               8   
2760   1428047  107993270  2202     1002565     85.394               2   
2763   1428047  107993270  2202     1002565   4490.000               8   
2767   1428047  107993270  2202     1002565   1879.000              32   
2818   1428047  108033603  2202     1002565   5104.000               7   
2823   1428047  108033603  2202     1002565     85.000               2   
2826   1428047  108033603  2202     1002565   4491.000               7   
2830   1428047  108033603  2202     1002565   1886.000              32   
2976   1451704  108073632  2202     1002565   5979.000               9   
2984   1451704  108073632  2202     1002565   3126.000              10   
4041   1520984  108045117  2202     10

In [12]:
df_IDW_2202.corr()

,Material,BATCHID,Quantity,Phase_duration,Phase_start_delay,Flowrate_KGMIN,Phase_overrun
Material,1.000000,-0.418475,0.342249,-0.010677,0.054867,0.183943,-0.037476
BATCHID,-0.418475,1.000000,-0.000991,0.149138,0.223400,-0.112222,0.152095
Quantity,0.342249,-0.000991,1.000000,-0.091609,-0.049696,0.872374,-0.154631
Phase_duration,-0.010677,0.149138,-0.091609,1.000000,0.645133,-0.405153,0.997584
Phase_start_delay,0.054867,0.223400,-0.049696,0.645133,1.000000,-0.224607,0.645603
Flowrate_KGMIN,0.183943,-0.112222,0.872374,-0.405153,-0.224607,1.000000,-0.455232
Phase_overrun,-0.037476,0.152095,-0.154631,0.997584,0.645603,-0.455232,1.000000


In [13]:
import pandas as pd
# Assuming df_filtered1 DataFrames are defined
# Create a new column 'QuantityRange' in df_filtered1 based on the 'Quantity' values
df_IDW_2202['QuantityRange'] = pd.cut(df_IDW_2202['Quantity'], bins=2, labels=['QuantityRange <= 5000', 'QuantityRange >= 5000'])


<ipython-input-13-c1e0f843bdc8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_IDW_2202['QuantityRange'] = pd.cut(df_IDW_2202['Quantity'], bins=2, labels=['QuantityRange <= 5000', 'QuantityRange >= 5000'])


In [14]:
# Or rename the existing DataFrame (rather than creating a copy)
df_IDW_2202.rename(columns={'Quantity': 'WaterQuantity','Phase_duration':'WaterPhase_duration','Phase_start_delay':'WaterPhase_start_delay','Flowrate_KGMIN':'WaterFlowrate_KGMIN','Phase_overrun':'WaterPhase_overrun','QuantityRange':'WaterQuantityRange'}, inplace=True)
print(df_IDW_2202)

      Material    BATCHID  Unit Material_ID  WaterQuantity  \
2002   1397022  107956670  2202     1002565       8623.000   
2012   1397022  107956670  2202     1002565       2065.000   
2755   1428047  107993270  2202     1002565       5101.000   
2760   1428047  107993270  2202     1002565         85.394   
2763   1428047  107993270  2202     1002565       4490.000   
2767   1428047  107993270  2202     1002565       1879.000   
2818   1428047  108033603  2202     1002565       5104.000   
2823   1428047  108033603  2202     1002565         85.000   
2826   1428047  108033603  2202     1002565       4491.000   
2830   1428047  108033603  2202     1002565       1886.000   
2976   1451704  108073632  2202     1002565       5979.000   
2984   1451704  108073632  2202     1002565       3126.000   
4041   1520984  108045117  2202     1002565      12478.000   
4047   1520984  108045117  2202     1002565       3994.000   
4845   1648637  107867810  2202     1002565       6067.000   
4851   1

C:\Users\User\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [15]:
df_IDW_2202.describe()

,Material,BATCHID,WaterQuantity,WaterPhase_duration,WaterPhase_start_delay,WaterFlowrate_KGMIN,WaterPhase_overrun
count,2.900000e+01,2.900000e+01,29.000000,29.000000,29.000000,29.000000,29.000000
mean,1.587341e+06,1.079837e+08,5074.013586,22.310345,1.620690,458.178148,14.620690
std,1.536906e+05,5.856128e+04,3681.340472,37.009751,5.010088,355.591735,37.119896
min,1.397022e+06,1.078678e+08,85.000000,2.000000,0.000000,3.717900,0.000000
25%,1.428047e+06,1.079567e+08,3126.000000,8.000000,0.000000,58.937500,0.000000
50%,1.648637e+06,1.079933e+08,4490.000000,12.000000,0.000000,561.250000,1.000000
75%,1.775253e+06,1.080336e+08,6064.000000,17.000000,1.000000,729.142900,10.000000
max,1.775253e+06,1.080736e+08,12478.000000,190.000000,21.000000,1039.833300,183.000000


In [16]:
df_IDW_2202.corr()

,Material,BATCHID,WaterQuantity,WaterPhase_duration,WaterPhase_start_delay,WaterFlowrate_KGMIN,WaterPhase_overrun
Material,1.000000,-0.418475,0.342249,-0.010677,0.054867,0.183943,-0.037476
BATCHID,-0.418475,1.000000,-0.000991,0.149138,0.223400,-0.112222,0.152095
WaterQuantity,0.342249,-0.000991,1.000000,-0.091609,-0.049696,0.872374,-0.154631
WaterPhase_duration,-0.010677,0.149138,-0.091609,1.000000,0.645133,-0.405153,0.997584
WaterPhase_start_delay,0.054867,0.223400,-0.049696,0.645133,1.000000,-0.224607,0.645603
WaterFlowrate_KGMIN,0.183943,-0.112222,0.872374,-0.405153,-0.224607,1.000000,-0.455232
WaterPhase_overrun,-0.037476,0.152095,-0.154631,0.997584,0.645603,-0.455232,1.000000


In [17]:
WaterQuantityRange = ['QuantityRange >= 5000']
df_Material_IDWater1= df_IDW_2202[df_IDW_2202['WaterQuantityRange'].isin(WaterQuantityRange)]
print(df_Material_IDWater1)

      Material    BATCHID  Unit Material_ID  WaterQuantity  \
2002   1397022  107956670  2202     1002565         8623.0   
4041   1520984  108045117  2202     1002565        12478.0   
7316   1775253  107899926  2202     1002565        11476.0   
7556   1775253  107964410  2202     1002565        11479.0   
7668   1775253  107999494  2202     1002565        11476.0   
7697   1775253  108026759  2202     1002565        11478.0   

      WaterPhase_duration  WaterPhase_start_delay  WaterFlowrate_KGMIN  \
2002                   10                       1             862.3000   
4041                   12                       3            1039.8333   
7316                   12                       0             956.3333   
7556                   12                       0             956.5833   
7668                   14                       0             819.7143   
7697                   12                       0             956.5000   

      WaterPhase_overrun     WaterQuantityRang

In [18]:
df_Material_IDWater1.describe()

,Material,BATCHID,WaterQuantity,WaterPhase_duration,WaterPhase_start_delay,WaterFlowrate_KGMIN,WaterPhase_overrun
count,6.000000e+00,6.000000e+00,6.000000,6.000000,6.000000,6.000000,6.000000
mean,1.669836e+06,1.079821e+08,11168.333333,12.000000,0.666667,931.877367,0.333333
std,1.679496e+05,5.287778e+04,1309.631500,1.264911,1.211060,78.601268,0.816497
min,1.397022e+06,1.078999e+08,8623.000000,10.000000,0.000000,819.714300,0.000000
25%,1.584551e+06,1.079586e+08,11476.000000,12.000000,0.000000,885.808325,0.000000
50%,1.775253e+06,1.079820e+08,11477.000000,12.000000,0.000000,956.416650,0.000000
75%,1.775253e+06,1.080199e+08,11478.750000,12.000000,0.750000,956.562475,0.000000
max,1.775253e+06,1.080451e+08,12478.000000,14.000000,3.000000,1039.833300,2.000000


In [19]:
WaterQuantityRange = ['QuantityRange <= 5000']
df_Material_IDWater2 = df_IDW_2202[df_IDW_2202['WaterQuantityRange'].isin(WaterQuantityRange)]
print(df_Material_IDWater2)

      Material    BATCHID  Unit Material_ID  WaterQuantity  \
2012   1397022  107956670  2202     1002565       2065.000   
2755   1428047  107993270  2202     1002565       5101.000   
2760   1428047  107993270  2202     1002565         85.394   
2763   1428047  107993270  2202     1002565       4490.000   
2767   1428047  107993270  2202     1002565       1879.000   
2818   1428047  108033603  2202     1002565       5104.000   
2823   1428047  108033603  2202     1002565         85.000   
2826   1428047  108033603  2202     1002565       4491.000   
2830   1428047  108033603  2202     1002565       1886.000   
2976   1451704  108073632  2202     1002565       5979.000   
2984   1451704  108073632  2202     1002565       3126.000   
4047   1520984  108045117  2202     1002565       3994.000   
4845   1648637  107867810  2202     1002565       6067.000   
4851   1648637  107867810  2202     1002565        145.000   
4853   1648637  107867810  2202     1002565       5722.000   
4912   1

In [20]:
df_Material_IDWater2.describe()

,Material,BATCHID,WaterQuantity,WaterPhase_duration,WaterPhase_start_delay,WaterFlowrate_KGMIN,WaterPhase_overrun
count,2.300000e+01,2.300000e+01,23.000000,23.000000,23.000000,23.000000,23.000000
mean,1.565821e+06,1.079841e+08,3484.191043,25.000000,1.869565,334.604439,18.347826
std,1.459860e+05,6.106010e+04,2021.589169,41.308155,5.594676,286.991953,41.028381
min,1.397022e+06,1.078678e+08,85.000000,2.000000,0.000000,3.717900,0.000000
25%,1.428047e+06,1.079605e+08,1975.500000,8.000000,0.000000,58.349300,0.000000
50%,1.520984e+06,1.079933e+08,3598.000000,10.000000,0.000000,276.769200,4.000000
75%,1.648637e+06,1.080336e+08,5102.500000,22.500000,0.500000,639.598200,16.000000
max,1.775253e+06,1.080736e+08,6067.000000,190.000000,21.000000,758.375000,183.000000


## Addition of Preservative 

In [21]:
Material_ID = ['1002874','1002818','1037802']
df_IDP4_2202=TANK_2202[TANK_2202['Material_ID'].isin(Material_ID)]
# Or rename the existing DataFrame (rather than creating a copy)
df_IDP4_2202.rename(columns={'Quantity': 'Pres4Quantity','Phase_duration':'Pres4Phase_duration','Phase_start_delay':'Pres4Phase_start_delay','Flowrate_KGMIN':'Pres4Flowrate_KGMIN','Phase_overrun':'Pres4Phase_overrun'}, inplace=True)
print(df_IDP4_2202)

      Material    BATCHID  Unit Material_ID  Pres4Quantity  \
2004   1397022  107956670  2202     1037802         19.446   
2005   1397022  107956670  2202     1002818        516.000   
2757   1428047  107993270  2202     1037802         17.065   
2758   1428047  107993270  2202     1002818         68.326   
2820   1428047  108033603  2202     1037802         17.090   
2821   1428047  108033603  2202     1002818         68.382   
2978   1451704  108073632  2202     1002874         12.620   
2979   1451704  108073632  2202     1002818         48.321   
4043   1520984  108045117  2202     1002874         20.010   
4044   1520984  108045117  2202     1002818         75.010   
4847   1648637  107867810  2202     1002874         15.287   
4848   1648637  107867810  2202     1002818         57.235   
4914   1648637  107978116  2202     1002874         15.264   
4915   1648637  107978116  2202     1002818         57.231   
7318   1775253  107899926  2202     1002874         27.107   
7319   1

C:\Users\User\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [52]:
#Aggregate data per tank
aggregated_df_IDP4_2202= df_IDP4_2202.groupby('Material').agg({
    'Pres4Phase_duration': 'mean',
    'Pres4Phase_overrun': 'sum',
    'Pres4Phase_start_delay':'sum',
    'Pres4Quantity':'sum',
    'Pres4Flowrate_KGMIN':'sum',
    
}).reset_index()

 #Print the aggregated DataFrame
print(aggregated_df_IDP4_2202)

   Material  Pres4Phase_duration  Pres4Phase_overrun  Pres4Phase_start_delay  \
0   1397022                12.50                14.0                       0   
1   1428047                12.00                33.0                       0   
2   1451704               105.50               204.0                       0   
3   1520984                13.00                21.0                       0   
4   1648637                15.00                39.0                       0   
5   1775253                 8.75                26.0                       1   

   Pres4Quantity  Pres4Flowrate_KGMIN  
0        535.446              65.6439  
1        170.863              40.3430  
2         60.941               9.7255  
3         95.020              38.3388  
4        145.017              12.4999  
5        591.622              77.7067  


In [22]:
df_IDP4_2202.describe()

,Material,BATCHID,Pres4Quantity,Pres4Phase_duration,Pres4Phase_start_delay,Pres4Flowrate_KGMIN,Pres4Phase_overrun
count,2.200000e+01,2.200000e+01,22.000000,22.000000,22.000000,22.000000,22.000000
mean,1.602190e+06,1.079853e+08,72.677682,20.000000,0.045455,11.102627,15.318182
std,1.561757e+05,6.023597e+04,106.283668,42.563118,0.213201,15.203514,43.156842
min,1.397022e+06,1.078678e+08,12.620000,2.000000,0.000000,0.061300,0.000000
25%,1.433961e+06,1.079586e+08,19.587000,6.000000,0.000000,1.747700,0.000000
50%,1.648637e+06,1.079933e+08,37.720000,8.000000,0.000000,5.107250,2.500000
75%,1.775253e+06,1.080319e+08,73.353000,12.500000,0.000000,15.791550,8.750000
max,1.775253e+06,1.080736e+08,516.000000,206.000000,1.000000,64.500000,204.000000


## Addition of GUM 

In [23]:
Material_ID = ['1461896','1254972','1031006','1243269']
df_IDGUM_2202=TANK_2202[TANK_2202['Material_ID'].isin(Material_ID)]
# Or rename the existing DataFrame (rather than creating a copy)
df_IDGUM_2202.rename(columns={'Quantity': 'GUMQuantity','Phase_duration':'GUMPhase_duration','Phase_start_delay':'GUMPhase_start_delay','Flowrate_KGMIN':'GUMFlowrate_KGMIN','Phase_overrun':'GUMPhase_overrun'}, inplace=True)
print(df_IDGUM_2202)

      Material    BATCHID  Unit Material_ID  GUMQuantity  GUMPhase_duration  \
2008   1397022  107956670  2202     1461896     3500.000                 45   
2010   1397022  107956670  2202     1254972      396.000                 15   
2761   1428047  107993270  2202     1461896     1500.000                 24   
2765   1428047  107993270  2202     1461896     2000.000                 56   
2824   1428047  108033603  2202     1461896     1500.000                 28   
2828   1428047  108033603  2202     1461896     2000.000                 10   
2981   1451704  108073632  2202     1461896     2000.000                 58   
2982   1451704  108073632  2202     1254972       19.350                414   
4045   1520984  108045117  2202     1461896     1500.000                155   
4849   1648637  107867810  2202     1461896     1000.000                462   
4850   1648637  107867810  2202     1254972      144.778                  5   
4916   1648637  107978116  2202     1461896     1000

C:\Users\User\anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [24]:
df_IDGUM_2202.describe()

,Material,BATCHID,GUMQuantity,GUMPhase_duration,GUMPhase_start_delay,GUMFlowrate_KGMIN,GUMPhase_overrun
count,1.700000e+01,1.700000e+01,17.000000,17.000000,17.000000,17.000000,17.000000
mean,1.566246e+06,1.079848e+08,1688.536941,170.117647,0.117647,40.431765,139.882353
std,1.507992e+05,6.273377e+04,1138.241710,270.826947,0.332106,46.655401,265.049496
min,1.397022e+06,1.078678e+08,19.350000,5.000000,0.000000,0.046700,0.000000
25%,1.428047e+06,1.079567e+08,1000.000000,24.000000,0.000000,9.677400,0.000000
50%,1.520984e+06,1.079933e+08,1500.000000,56.000000,0.000000,32.258100,11.000000
75%,1.648637e+06,1.080336e+08,3000.000000,155.000000,0.000000,44.776100,127.000000
max,1.775253e+06,1.080736e+08,3500.000000,1030.000000,1.000000,200.000000,974.000000


In [25]:
df_IDGUM_2202.corr()

,Material,BATCHID,GUMQuantity,GUMPhase_duration,GUMPhase_start_delay,GUMFlowrate_KGMIN,GUMPhase_overrun
Material,1.000000,-0.395928,0.280685,0.435907,-0.344937,-0.378467,0.410501
BATCHID,-0.395928,1.000000,0.063707,-0.303966,0.171703,0.228932,-0.308162
GUMQuantity,0.280685,0.063707,1.000000,0.221054,0.020324,0.226072,0.161820
GUMPhase_duration,0.435907,-0.303966,0.221054,1.000000,-0.212798,-0.436386,0.997770
GUMPhase_start_delay,-0.344937,0.171703,0.020324,-0.212798,1.000000,0.732664,-0.198641
GUMFlowrate_KGMIN,-0.378467,0.228932,0.226072,-0.436386,0.732664,1.000000,-0.434507
GUMPhase_overrun,0.410501,-0.308162,0.161820,0.997770,-0.198641,-0.434507,1.000000


In [43]:
#Aggregate data per tank
aggregated_df_IDGUM_2202= df_IDGUM_2202.groupby('BATCHID').agg({
    'GUMPhase_duration': 'mean',
    'GUMPhase_overrun': 'sum',
    'GUMPhase_start_delay':'sum',
    'GUMQuantity':'sum',
    'GUMFlowrate_KGMIN':'sum',
    
}).reset_index()

 #Print the aggregated DataFrame
print(aggregated_df_IDGUM_2202)

      BATCHID  GUMPhase_duration  GUMPhase_overrun  GUMPhase_start_delay  \
0   107867810              233.5             436.0                     0   
1   107899926             1030.0             974.0                     0   
2   107956670               30.0               0.0                     0   
3   107964410               67.0              11.0                     0   
4   107978116               18.5               5.0                     0   
5   107993270               40.0              19.0                     1   
6   107999494              419.0             363.0                     0   
7   108026759               67.0              11.0                     0   
8   108033603               19.0               0.0                     1   
9   108045117              155.0             127.0                     0   
10  108073632              236.0             432.0                     0   

    GUMQuantity  GUMFlowrate_KGMIN  
0      1144.778            31.1201  
1      3000.0

## Colour Addition - There are only 3 batches that have colour ingredients included that were produced in Tank 2202

In [27]:
#Material_ID = ['3026582','3010810','3028657']
#df_IDCOL_2202=TANK_2202[TANK_2202['Material_ID'].isin(Material_ID)]
#print(df_IDCOL_2202)

In [28]:
# Or rename the existing DataFrame (rather than creating a copy)
#df_IDCOL_2202.rename(columns={'Quantity': 'COLQuantity','Phase_duration':'COLPhase_duration','Phase_start_delay':'COLPhase_start_delay','Flowrate_KGMIN':'COLFlowrate_KGMIN','Phase_overrun':'COLPhase_overrun'}, inplace=True)

In [29]:
#Aggregate data per tank
#aggregated_df_IDCOL_2202 = df_IDCOL_2202.groupby('Material').agg({
    #'COLPhase_duration': 'mean',
    #'COLPhase_overrun': 'sum',
   # 'COLPhase_start_delay':'sum',
   # 'COLQuantity':'sum',
   # 'COLFlowrate_KGMIN':'sum',
   # 'Target_Phase_duration':'sum',
   # 'Target_Flowrate':'sum'
#}).reset_index()

 #Print the aggregated DataFrame
#print(aggregated_df_IDCOL_2202)

In [42]:
# Pandas join on columns
frames = [df_IDW_2202,df_IDGUM_2202,df_IDP4_2202]
df2 = pd.merge(df_IDW_2202,df_IDGUM_2202, how='right', right_on = ['BATCHID'])
print(df2)

MergeError: Must pass "left_on" OR "left_index".

In [34]:
#frames = [df_IDW_2202,df_IDGUM_2202,df_IDP4_2202]
result = pd.merge(df_IDW_2202,df_IDGUM_2202,how ='outer')
print(result)

    Material    BATCHID  Unit Material_ID  WaterQuantity  WaterPhase_duration  \
0    1397022  107956670  2202     1002565       8623.000                 10.0   
1    1397022  107956670  2202     1002565       2065.000                 13.0   
2    1428047  107993270  2202     1002565       5101.000                  8.0   
3    1428047  107993270  2202     1002565         85.394                  2.0   
4    1428047  107993270  2202     1002565       4490.000                  8.0   
5    1428047  107993270  2202     1002565       1879.000                 32.0   
6    1428047  108033603  2202     1002565       5104.000                  7.0   
7    1428047  108033603  2202     1002565         85.000                  2.0   
8    1428047  108033603  2202     1002565       4491.000                  7.0   
9    1428047  108033603  2202     1002565       1886.000                 32.0   
10   1451704  108073632  2202     1002565       5979.000                  9.0   
11   1451704  108073632  220

In [ ]:
frames = [df_IDW_2202,df_IDGUM_2202,df_IDP4_2202]

result = pd.concat(frames)
print(result)

In [ ]:
numerical_columns = result.select_dtypes(include=['float64', 'int64']).columns

for col in numerical_columns:
    sns.histplot(data=result, x=col, bins=30, kde=True)
    plt.title(f"Distribution of {col}")
    plt.show()

In [ ]:
result.describe()

In [ ]:
#always remember your magic function if using Jupyter
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
sns.heatmap(result)
plt.show()

In [ ]:
sns.regplot(x='Pres3Phase_duration',y='Pres3Phase_overrun',data=result)

## Effect of Agitation after ingredient addition

In [ ]:
#Select all the rows where the material is in the list below
Instruction_Step = ['STEP1_AGITATION','STEP2_AGITATION']
df_Agitation = TANK_2202[TANK_2202['Instruction_Step'].isin(Instruction_Step)]
#print(df_Agitation)

In [ ]:
#Aggregate data per tank
aggregated_df_Agitation = df_Agitation.groupby('Material').agg({
    'Phase_duration': 'mean',
    'Phase_overrun': 'sum',
    'Phase_start_delay':'sum',
    'Quantity':'sum',
    'Flowrate_KGMIN':'sum',
    'Target_Phase_duration':'sum',
    'Target_Flowrate':'sum'
}).reset_index()

 #Print the aggregated DataFrame
print(aggregated_df_Agitation)